# OpenSky API Demo
The purpose of this demo is to test the [OpenSky API](https://github.com/openskynetwork/opensky-api) for retrieving flight data and converting to CSV format for downstream processing.

## Setup
Follow installation intructions at https://github.com/openskynetwork/opensky-api?tab=readme-ov-file#installation

In [6]:
!python --version

Python 3.12.4


In [9]:
!if [ ! -d ~/opensky-api ]; then git clone https://github.com/openskynetwork/opensky-api.git ~/opensky-api; fi
!python -m pip install -q -e ~/opensky-api/python

  DEPRECATION: Legacy editable install of opensky-api==1.3.0 from file:///home/ballen/opensky-api/python (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


**NOTE:** Restart kernel if this is the first time installing opensky-api

In [1]:
!wget https://raw.githubusercontent.com/ip2location/ip2location-iata-icao/master/iata-icao.csv -O iata-icao.csv

--2024-09-17 13:06:03--  https://raw.githubusercontent.com/ip2location/ip2location-iata-icao/master/iata-icao.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 693127 (677K) [text/plain]
Saving to: ‘iata-icao.csv’

iata-icao.csv       100%[===================>] 676.88K  --.-KB/s    in 0.005s  

2024-09-17 13:06:03 (131 MB/s) - ‘iata-icao.csv’ saved [693127/693127]



In [154]:
!python -m pip install -q pandas folium

## Imports

In [155]:
from opensky_api import OpenSkyApi, FlightData, FlightTrack, Waypoint
import pandas as pd
import folium
from folium import Popup
import datetime
from typing import List

## Configurations

In [4]:
api = OpenSkyApi()

In [5]:
def print_null_values_report(df):
    # Inspect percent of null values for each column across the dataset
    missing_count = df.isna().sum()  # Count of missing values
    total_count = len(df)  # Total number of rows in the DataFrame
    percentage_missing = (missing_count / total_count) * 100  # Calculate percentage
    
    # Combine count and percentage into a new DataFrame
    missing_info = pd.DataFrame({'Missing Count': missing_count, 'Percentage Missing': percentage_missing})
    
    # Display the result
    print(missing_info)

In [6]:
def to_epoch_time(dt: datetime.datetime) -> int:
    return int(dt.timestamp())

In [120]:
def to_timestamp(epoch_time: int) -> datetime.datetime:
    return datetime.datetime.fromtimestamp(epoch_time)

In [7]:
def flight_data_to_pandas(data: List[FlightData]) -> pd.DataFrame:
    """
    Convert flight data response object to pandas datafrme.

    Args:
        data (List[FlightData]): List of flight data objects.

    Returns:
        pd.DataFrame: Pandas dataframe representing the original list of flight data objects.
    """
    rows = []
    cols = None
    for record in data:
        row = []
        cols = record.keys
        for attribute in cols:
            item = record.__getattribute__(attribute)
            if isinstance(item, str):
                item = item.strip()
            row.append(item)
        rows.append(row)

    return pd.DataFrame(rows, columns=cols)

In [169]:
def flight_track_to_pandas(track: FlightTrack) -> pd.DataFrame:
    """
    Convert flight track response object to pandas dataframe.

    Args:
        track (FlightTrack): Flight track data object.

    Returns:
        pd.DataFrame: Pandas dataframe representing the original flight track data object with expanded path of waypoints.
    """
    rows = []
    cols = None
    for record in track.path:
        waypoint = Waypoint(record)
        cols = waypoint.keys
        row = [ track.icao24, track.startTime, track.endTime, track.callsign ]
        for attribute in cols:
            item = waypoint.__getattribute__(attribute)
            if isinstance(item, str):
                item = item.strip()
            row.append(item)
        rows.append(row)
    
    return pd.DataFrame(rows, columns=['icao24', 'startTime', 'endTime', 'callsign', *cols])


## Demo
**Documentation:** https://openskynetwork.github.io/opensky-api/python.html

### Get States
https://openskynetwork.github.io/opensky-api/python.html#opensky_api.OpenSkyApi.get_states

In [8]:
s = api.get_states()
print(s)

{   'states': [   StateVector(dict_values(['ab1644', 'UAL2723 ', 'United States', 1726592559, 1726592559, -94.2546, 30.7359, 6614.16, False, 218.81, 227.38, -6.18, None, 7894.32, '3640', False, 0, 0])),
                  StateVector(dict_values(['a2e5ec', 'SKW4119 ', 'United States', 1726592696, 1726592703, -112.0099, 33.4334, None, True, 1.03, 284.06, None, None, None, None, False, 0, 0])),
                  StateVector(dict_values(['aa56da', 'UAL1620 ', 'United States', 1726592780, 1726592780, -109.6799, 39.7975, 10363.2, False, 195.14, 257.98, -0.33, None, 10629.9, '3102', False, 0, 0])),
                  StateVector(dict_values(['a928b0', 'N69DH   ', 'United States', 1726592780, 1726592780, -117.2078, 34.4277, 10972.8, False, 169.46, 286.4, 0, None, 11407.14, None, False, 0, 0])),
                  StateVector(dict_values(['4b1812', 'SWR262C ', 'Switzerland', 1726592780, 1726592780, 5.017, 45.4441, 10721.34, False, 192.1, 55.03, -4.55, None, 11010.9, '1000', False, 0, 0])),
      

### Get Arrivals/Departures by Airport
**NOTE:** All airports should be entered as ICAO format.

Download ICAO airports list from https://github.com/ip2location/ip2location-iata-icao/blob/master/iata-icao.csv

#### Load Airports

In [9]:
airports = pd.read_csv('iata-icao.csv')
airports.head()

,country_code,region_name,iata,icao,airport,latitude,longitude
0,AE,Abu Zaby,AAN,OMAL,Al Ain International Airport,24.2617,55.6092
1,AE,Abu Zaby,AUH,OMAA,Abu Dhabi International Airport,24.4330,54.6511
2,AE,Abu Zaby,AYM,NaN,Yas Island Seaplane Base,24.4670,54.6103
3,AE,Abu Zaby,AZI,OMAD,Al Bateen Executive Airport,24.4283,54.4581
4,AE,Abu Zaby,DHF,OMAM,Al Dhafra Air Base,24.2482,54.5477


In [10]:
# Analyze shape and number of unique ICAO values
airports.shape, airports['icao'].nunique()

((8946, 7), 7801)

In [11]:
# Inspect null data
print_null_values_report(airports)

              Missing Count  Percentage Missing
country_code             32            0.357702
region_name               0            0.000000
iata                      0            0.000000
icao                   1145           12.799016
airport                   0            0.000000
latitude                  0            0.000000
longitude                 0            0.000000


In [12]:
# Inspect BWI airport
airports[airports['iata'] == 'BWI']

,country_code,region_name,iata,icao,airport,latitude,longitude
7620,US,Maryland,BWI,KBWI,Baltimore/Washington International Airport,39.1753,-76.6683


In [102]:
start_datetime = datetime.datetime.strptime('2024-08-08 09:55:00', '%Y-%m-%d %H:%M:%S') # 2024-08-08 09:55:00 (EST), 2024-08-08 13:55:00 (UTC)
end_datetime = datetime.datetime.strptime('2024-08-20 16:35:00', '%Y-%m-%d %H:%M:%S')   # 2024-08-20 16:35:00 (EST), 2024-08-20 20:35:00 (UTC)

In [103]:
to_epoch_time(start_datetime + datetime.timedelta(days=1))

1723211700

#### Arrivals
https://openskynetwork.github.io/opensky-api/python.html#opensky_api.OpenSkyApi.get_arrivals_by_airport

In [18]:
help(api.get_arrivals_by_airport)

Help on method get_arrivals_by_airport in module opensky_api:

get_arrivals_by_airport(airport, begin, end) method of opensky_api.OpenSkyApi instance
    Retrieves flights for a certain airport which arrived within a given time interval [begin, end].

    :param str airport: ICAO identier for the airport.
    :param int begin: Start of time interval to retrieve flights for as Unix time (seconds since epoch).
    :param int end: End of time interval to retrieve flights for as Unix time (seconds since epoch).
    :return: list of FlightData objects if request was successful, None otherwise..
    :rtype: FlightData | None



In [104]:
# Get arrivals for BWI
# NOTE: Occasionally timeout errors may occur when there is high volume requests. Need to wait and try again.
# NOTE: Time interval must be smaller than 7 days
arrivals = api.get_arrivals_by_airport(
    airport="KBWI", 
    begin=to_epoch_time(start_datetime), 
    end=to_epoch_time(start_datetime + datetime.timedelta(hours=1)),
)

In [105]:
len(arrivals)

20

In [106]:
arrivals[0]

FlightData(dict_values(['a2c9ec', 1723123956, 'KMDW', 1723128766, 'KBWI', 'SWA1751 ', 1658, 199, 2444, 31, 0, 7]))

In [107]:
arrivals[0].keys

['icao24',
 'firstSeen',
 'estDepartureAirport',
 'lastSeen',
 'estArrivalAirport',
 'callsign',
 'estDepartureAirportHorizDistance',
 'estDepartureAirportVertDistance',
 'estArrivalAirportHorizDistance',
 'estArrivalAirportVertDistance',
 'departureAirportCandidatesCount',
 'arrivalAirportCandidatesCount']

In [108]:
arrivals_df = flight_data_to_pandas(arrivals)
arrivals_df.head()

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
0,a2c9ec,1723123956,KMDW,1723128766,KBWI,SWA1751,1658.0,199.0,2444,31,0,7
1,ab64b8,1723123430,KORD,1723128518,KBWI,AAL2940,3063.0,221.0,2633,46,2,7
2,a5ee51,1723121282,KTPA,1723128189,KBWI,SWA4696,992.0,91.0,2407,31,2,7
3,a260ed,1723123688,KIND,1723127966,KBWI,SWA4253,3330.0,442.0,2026,16,2,7
4,a6d932,1723121900,None,1723127486,KBWI,GRB443,NaN,NaN,2365,39,0,7


#### Departures
https://openskynetwork.github.io/opensky-api/python.html#opensky_api.OpenSkyApi.get_departures_by_airport

In [109]:
# Get departures for BWI
# NOTE: Time interval must be smaller than 7 days
departures = api.get_departures_by_airport(
    airport="KBWI", 
    begin=to_epoch_time(start_datetime), 
    end=to_epoch_time(start_datetime + datetime.timedelta(hours=2)) 
)

In [110]:
len(departures)

55

In [111]:
departures_df = flight_data_to_pandas(departures)
departures_df.head()

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
0,abb624,1723132331,KBWI,1723135197,KCLE,SWA2925,1010,69,4923.0,246.0,1,15
1,a260ed,1723132278,KBWI,1723134921,KBDL,SWA4253,1041,107,11734.0,495.0,1,11
2,a62863,1723132171,KBWI,1723135008,KALB,SWA1278,1082,85,5000.0,149.0,1,10
3,a2c9ec,1723132130,KBWI,1723135749,KCMH,SWA102,952,54,6059.0,277.0,1,16
4,ac5fad,1723131998,KBWI,1723139571,KMSP,DAL2884,1234,85,1740.0,35.0,1,8


In [81]:
airports[airports['iata'] == 'HOU']

,country_code,region_name,iata,icao,airport,latitude,longitude
8349,US,Texas,HOU,KHOU,William P. Hobby Airport,29.6454,-95.2789


In [112]:
# icao24=abe501, callsign=SWA3751
departures_df[departures_df['estArrivalAirport'] == 'KHOU']

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
38,abe501,1723127115,KBWI,1723136143,KHOU,SWA3751,1168,92,7273.0,366.0,1,8


### Get Flights From Interval
https://openskynetwork.github.io/opensky-api/python.html#opensky_api.OpenSkyApi.get_flights_from_interval

In [30]:
# NOTE: Time interval must be smaller than 2 hours
flights = api.get_flights_from_interval(
    begin=to_epoch_time(start_datetime), 
    end=to_epoch_time(start_datetime + datetime.timedelta(hours=2))
)

In [31]:
len(flights)

6441

In [33]:
flights[0]

FlightData(dict_values(['78206e', 1723124068, None, 1723127245, None, 'CSZ9815 ', None, None, None, None, 0, 0]))

In [34]:
flights_df = flight_data_to_pandas(flights)
flights_df.head()

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
0,78206e,1723124068,None,1723127245,None,CSZ9815,NaN,NaN,NaN,NaN,0,0
1,781b95,1723124950,None,1723128346,None,CDC8900,NaN,NaN,NaN,NaN,0,0
2,80140c,1723125185,VOMM,1723126758,None,IGO37E,2075.0,189.0,NaN,NaN,1,0
3,8a03ac,1723125378,None,1723128952,None,CTV442,NaN,NaN,NaN,NaN,0,0
4,a4e11a,1723125701,KSLC,1723126722,None,N413UU,2593.0,113.0,NaN,NaN,1,0


In [39]:
flights_df[flights_df['estDepartureAirport'] == 'KBWI'].head()

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
273,ac08fa,1723125001,KBWI,1723129439,KMLJ,VTE3102,865.0,107.0,3919.0,2221.0,1,2
1583,a4b9c7,1723127490,KBWI,1723130548,KRDU,SWA3354,955.0,54.0,3254.0,172.0,1,7
2023,ac210e,1723125458,KBWI,1723129971,05IN,SWA2947,1033.0,85.0,1966.0,804.0,1,34
2238,a51f0f,1723126463,KBWI,1723129104,KISP,SWA901,939.0,46.0,2911.0,46.0,1,2
2311,aab169,1723125277,KBWI,1723129324,KPWM,SWA1572,7722.0,900.0,1925.0,38.0,1,12


### Get Flights by Aircraft
https://openskynetwork.github.io/opensky-api/python.html#opensky_api.OpenSkyApi.get_flights_by_aircraft

In [35]:
flights_by_aircraft = api.get_flights_by_aircraft(
    icao24='78206e',
    begin=to_epoch_time(start_datetime),
    end=to_epoch_time(end_datetime)
)

In [36]:
len(flights_by_aircraft)

18

In [38]:
flights_by_aircraft_df = flight_data_to_pandas(flights_by_aircraft)
flights_by_aircraft_df.head()

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
0,78206e,1724141238,None,1724152049,None,CSZ9638,NaN,NaN,None,None,0,0
1,78206e,1724118456,ZGSZ,1724127486,None,CSZ9637,3814.0,468.0,None,None,0,0
2,78206e,1724041068,None,1724042030,None,CSZ9802,NaN,NaN,None,None,0,0
3,78206e,1724024763,None,1724028129,None,CSZ9801,NaN,NaN,None,None,0,0
4,78206e,1723997941,None,1724001256,None,CSZ9918,NaN,NaN,None,None,0,0


### Get Flight Tracks by Aircraft
https://openskynetwork.github.io/opensky-api/python.html#opensky_api.OpenSkyApi.get_track_by_aircraft

In [125]:
past_datetime = datetime.datetime.now() - datetime.timedelta(days=30)

In [126]:
# Get flights from 30 days in the past (limit)
historic_flights = api.get_flights_from_interval(
    begin=to_epoch_time(past_datetime), 
    end=to_epoch_time(past_datetime + datetime.timedelta(hours=2))
)

In [127]:
historic_flights_df = flight_data_to_pandas(historic_flights)
historic_flights_df.head()

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
0,a5fa9f,1724007400,KEWR,1724009038,KEWR,N4846,5510.0,101.0,5951.0,70.0,1,5
1,a811af,1724009409,KHYA,1724011012,KOWD,N619RL,1531.0,196.0,311.0,14.0,0,7
2,a9a4b3,1724011606,KOWD,1724012923,KMVY,N720MV,231.0,0.0,2320.0,93.0,1,4
3,a9a4b3,1724007873,KMVY,1724008889,KOWD,N720MV,2178.0,200.0,379.0,0.0,2,6
4,aa3681,1724007101,KC29,1724010869,KC29,N75706,327.0,8.0,158.0,8.0,1,19


In [130]:
historic_flight_id, start_epoch, end_epoch = historic_flights_df[['icao24', 'firstSeen', 'lastSeen']].values[0].tolist()
print(f'historic_flight_id = {historic_flight_id}')
print(f'start_epoch        = {start_epoch}')
print(f'end_epoch          = {end_epoch}')

historic_flight_id = a5fa9f
start_epoch        = 1724007400
end_epoch          = 1724009038


In [131]:
to_timestamp(start_epoch), to_timestamp(end_epoch)

(datetime.datetime(2024, 8, 18, 14, 56, 40),
 datetime.datetime(2024, 8, 18, 15, 23, 58))

In [146]:
# NOTE: It is not possible to access flight tracks from more than 30 days in the past.
flight_track = api.get_track_by_aircraft(
    icao24=historic_flight_id,
    t=end_epoch
)

In [161]:
flight_track_points_df = flight_track_to_pandas(flight_track)
flight_track_points_df.head()

,icao24,startTime,endTime,callsign,time,latitude,longitude,baro_altitude,true_track,on_ground
0,a5fa9f,1.724007e+09,1.724009e+09,N4846,1724007400,40.7259,-74.1204,0,194,False
1,a5fa9f,1.724007e+09,1.724009e+09,N4846,1724007404,40.7245,-74.1207,0,193,False
2,a5fa9f,1.724007e+09,1.724009e+09,N4846,1724007405,40.7242,-74.1208,0,190,False
3,a5fa9f,1.724007e+09,1.724009e+09,N4846,1724007408,40.7233,-74.1211,0,188,False
4,a5fa9f,1.724007e+09,1.724009e+09,N4846,1724007410,40.7226,-74.1212,0,187,False


In [162]:
flight_track_points_df.shape

(612, 10)

#### Plot Track Points on Map

In [168]:
# Create a base map centered at the midpoint of the flight track
map_center = [flight_track_points_df['latitude'].mean(), flight_track_points_df['longitude'].mean()]
m = folium.Map(
    location=map_center, 
    zoom_start=12,
    tiles="CartoDB dark_matter",
    height="60%"
)

# Plot each point with small circle markers and dynamic popup text
for _, row in flight_track_points_df.iterrows():
    # Create dynamic popup text based on the dataframe's columns
    popup_text = '<br>'.join([f"{col}: {row[col]}" for col in flight_track_points_df.columns])
    
    # Add a small circle marker with a popup
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=Popup(popup_text, max_width=300)
    ).add_to(m)

# Add green polyline for the flight track
flight_path = flight_track_points_df[['latitude', 'longitude']].values.tolist()
folium.PolyLine(flight_path, color="green", weight=2.5, opacity=1).add_to(m)
m